## Import các thư viện cần dùng

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!apt update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
# !wget -q http://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar -xvf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
57 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [ ]:
!pip install surprise

In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from surprise import Reader, Dataset, SVD, SVDpp, NMF, SlopeOne, KNNBasic, KNNBaseline, KNNWithMeans, KNNWithZScore, CoClustering, BaselineOnly
from surprise.model_selection.validation import cross_validate

In [ ]:
spark = SparkSession.builder.appName('BigData_Suprise').getOrCreate()

## 1 . Đọc và xử lý dữ liệu

### File output.csv

In [ ]:
Evaluate = spark.read.csv('/content/drive/MyDrive/DL07_K299_TranDangDiep/Project2/data/output.csv',header=True,inferSchema=True)

In [ ]:
Evaluate.printSchema()

root
 |-- ma_khach_hang: integer (nullable = true)
 |-- noi_dung_binh_luan: string (nullable = true)
 |-- ngay_binh_luan: string (nullable = true)
 |-- gio_binh_luan: timestamp (nullable = true)
 |-- so_sao: integer (nullable = true)
 |-- ma_san_pham: integer (nullable = true)
 |-- length: integer (nullable = true)
 |-- sentiment: string (nullable = true)
 |-- positive_count: integer (nullable = true)
 |-- negative_count: integer (nullable = true)



In [ ]:
Evaluate.show(5)

+-------------+--------------------+--------------+-------------------+------+-----------+------+---------+--------------+--------------+
|ma_khach_hang|  noi_dung_binh_luan|ngay_binh_luan|      gio_binh_luan|so_sao|ma_san_pham|length|sentiment|positive_count|negative_count|
+-------------+--------------------+--------------+-------------------+------+-----------+------+---------+--------------+--------------+
|          443|dễ_dàng thoải_mái...|    29/04/2023|2024-11-29 17:06:00|     5|  308500015|    48| positive|             1|             0|
|         1030|dãng thoải_mái th...|    30/04/2023|2024-11-29 15:04:00|     5|  308500015|    45| positive|             1|             0|
|          689|        thích hasaki|    30/04/2023|2024-11-29 18:34:00|     5|  422216594|    41| positive|             1|             0|
|         2519|khả_năng sạch tốt...|    17/07/2022|2024-11-29 13:48:00|     5|  204100075|   378| positive|             1|             1|
|          402|sữa rửa tốt sạch ..

#### Kiếm tra Null ,nan

In [ ]:
Evaluate.select([count(when(col(c).isNull(),c)).alias(c) for c in Evaluate.columns]).toPandas().T

,0
ma_khach_hang,0
noi_dung_binh_luan,1287
ngay_binh_luan,0
gio_binh_luan,0
so_sao,0
ma_san_pham,0
length,0
sentiment,0
positive_count,0
negative_count,0


In [ ]:
Evaluate = Evaluate.na.drop()

In [ ]:
Evaluate.select([count(when(col(c).isNull(),c)).alias(c) for c in Evaluate.columns]).toPandas().T

,0
ma_khach_hang,0
noi_dung_binh_luan,0
ngay_binh_luan,0
gio_binh_luan,0
so_sao,0
ma_san_pham,0
length,0
sentiment,0
positive_count,0
negative_count,0


#### Kiếm tra dữ liệu trùng

In [ ]:
Evaluate.count()

19387

In [ ]:
Evaluate.distinct().count()

19387

### File Product.csv

In [ ]:
Product = spark.read.csv('/content/drive/MyDrive/DL07_K299_TranDangDiep/Project2/data/Product_ml.csv',
                    header=True,
                    inferSchema=True,
                    multiLine=True,
                    escape='"')

In [ ]:
Product.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ma_san_pham: integer (nullable = true)
 |-- ten_san_pham: string (nullable = true)
 |-- gia_ban: integer (nullable = true)
 |-- gia_goc: double (nullable = true)
 |-- phan_loai: string (nullable = true)
 |-- mo_ta: string (nullable = true)
 |-- diem_trung_binh: double (nullable = true)
 |-- Content: string (nullable = true)
 |-- Content_wt: string (nullable = true)
 |-- content_gem_re: string (nullable = true)



In [ ]:
Product.show(5)

+---+-----------+--------------------+-------+--------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+
|_c0|ma_san_pham|        ten_san_pham|gia_ban| gia_goc|           phan_loai|               mo_ta|diem_trung_binh|             Content|          Content_wt|      content_gem_re|
+---+-----------+--------------------+-------+--------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+
|  0|  318900012|Nước Hoa Hồng Kla...| 209000|435000.0|2x180ml,180ml,Khô...|Nước Hoa Hồng Kla...|            4.8|Nước Hoa Hồng Kla...|Nước Hoa_Hồng Kla...|['hoa_hồng', 'kla...|
|  1|  205100137|Nước Tẩy Trang L'...| 147000|229000.0|2x400ml,95ml,400m...|Nước Tẩy Trang L'...|            4.7|Nước Tẩy Trang L'...|Nước Tẩy_Trang L'...|['tẩy_trang', "l'...|
|  2|  422208973|Sữa Rửa Mặt CeraV...| 343000|455000.0|88ml,236ml,473ml,...|Sữa Rửa Mặt Cerav...|            4.9|Sữ

#### Xóa cột _c0

In [ ]:
Product = Product.drop('_c0')

#### kiếm tra null , nan

In [ ]:
Product.select([count(when(col(c).isNull(),c)).alias(c) for c in Product.columns]).toPandas().T

,0
ma_san_pham,0
ten_san_pham,0
gia_ban,0
gia_goc,0
phan_loai,0
mo_ta,0
diem_trung_binh,0
Content,0
Content_wt,0
content_gem_re,0


In [ ]:
Product.select([count(when(isnan(c),c)).alias(c) for c in Product.columns]).toPandas().T

,0
ma_san_pham,0
ten_san_pham,0
gia_ban,0
gia_goc,0
phan_loai,0
mo_ta,0
diem_trung_binh,0
Content,0
Content_wt,0
content_gem_re,0


#### Kiếm tra dữ liệu trùng

In [ ]:
Product.count()

1186

In [ ]:
Product.distinct().count()

1186

## 2. Merge và Chọn ra các cột cần thiết

In [ ]:
merged_df = Evaluate.join(Product.select("ma_san_pham", "ten_san_pham"), on="ma_san_pham", how="left")

In [ ]:
df_sub = merged_df.select(['ma_khach_hang','so_sao','ma_san_pham','ten_san_pham'])

In [ ]:
df_sub.count()

19387

In [ ]:
# Lọc người dùng có ít đánh giá
filtered_data  = df_sub.groupBy("ma_khach_hang").count().filter(col("count") <= 3)

In [ ]:
filtered_data.show(5)

+-------------+-----+
|ma_khach_hang|count|
+-------------+-----+
|         2142|    2|
|         3749|    1|
|         3997|    1|
|         3794|    2|
|          471|    2|
+-------------+-----+
only showing top 5 rows



In [ ]:
df_sub = df_sub.join(filtered_data,on = 'ma_khach_hang', how = 'left_anti')

In [ ]:
df_sub.show(5)

+-------------+------+-----------+--------------------+
|ma_khach_hang|so_sao|ma_san_pham|        ten_san_pham|
+-------------+------+-----------+--------------------+
|         2659|     5|  422203809|Combo 3 Bông Tẩy ...|
|          833|     5|  422204028|Kem Chống Nắng L'...|
|         2366|     5|  422205164|Hộp 10 Miếng Mặt ...|
|         1088|     5|  422211183|Nước Tẩy Trang Ga...|
|         2659|     2|  422204659|Mặt Nạ Nghệ Hưng ...|
+-------------+------+-----------+--------------------+
only showing top 5 rows



#### kiếm tra dữ liệu bị Null

In [ ]:
df_sub.select([count(when(col(c).isNull(),c)).alias(c) for c in df_sub.columns]).toPandas().T

,0
ma_khach_hang,0
so_sao,0
ma_san_pham,0
ten_san_pham,20


In [ ]:
df_sub = df_sub.na.drop()

In [ ]:
df_sub.select([count(when(col(c).isNull(),c)).alias(c) for c in df_sub.columns]).toPandas().T

,0
ma_khach_hang,0
so_sao,0
ma_san_pham,0
ten_san_pham,0


#### Kiếm tra dữ liệu bị trùng

In [ ]:
df_sub.count()

15250

In [ ]:
df_sub.distinct().count()

15055

In [ ]:
df_sub = df_sub.dropDuplicates()
df_sub.count()

15055

### Kiếm tra kiểu dữ liệu

In [ ]:
df_sub.dtypes

[('ma_khach_hang', 'int'),
 ('so_sao', 'int'),
 ('ma_san_pham', 'int'),
 ('ten_san_pham', 'string')]

In [ ]:
df_sub.show(5, truncate=True)

+-------------+------+-----------+--------------------+
|ma_khach_hang|so_sao|ma_san_pham|        ten_san_pham|
+-------------+------+-----------+--------------------+
|         2659|     5|  422203809|Combo 3 Bông Tẩy ...|
|          833|     5|  422204028|Kem Chống Nắng L'...|
|         2366|     5|  422205164|Hộp 10 Miếng Mặt ...|
|         1088|     5|  422211183|Nước Tẩy Trang Ga...|
|         2659|     2|  422204659|Mặt Nạ Nghệ Hưng ...|
+-------------+------+-----------+--------------------+
only showing top 5 rows



### thông tin số dòng của các cột

In [ ]:
users = df_sub.select("ma_khach_hang").distinct().count()
products = df_sub.select("ma_san_pham").distinct().count()
numerator = df_sub.count()

In [ ]:
display(numerator, users, products)

15055

1784

657

### Sparsity

In [ ]:
sparsity = 1 - (numerator*1.0 / (users*products))
print ("Sparsity: "), sparsity

Sparsity: 


(None, 0.9871554013009262)

### chuyển Dataframe Spark thành Dataframe Pandas

In [ ]:
df_sub = df_sub.toPandas()

In [ ]:
df_sub

,ma_khach_hang,so_sao,ma_san_pham,ten_san_pham
0,2659,5,422203809,Combo 3 Bông Tẩy Trang Ipek 150 Miếng
1,833,5,422204028,Kem Chống Nắng L'Oreal Mịn Nhẹ Kiềm Dầu Thoáng...
2,2366,5,422205164,Hộp 10 Miếng Mặt Nạ Naruko Ý Dĩ Nhân Đỏ Sáng D...
3,1088,5,422211183,Nước Tẩy Trang Garnier Dành Cho Da Dầu Và Mụn ...
4,2659,2,422204659,Mặt Nạ Nghệ Hưng Yên Cocoon Giúp Da Rạng Rỡ Mị...
...,...,...,...,...
15050,1138,5,422207866,Sữa Rửa Mặt Cetaphil Dịu Lành Cho Da Nhạy Cảm ...
15051,1138,5,422207865,Sữa Rửa Mặt Cetaphil Dịu Lành Cho Da Nhạy Cảm ...
15052,1413,5,422217029,Bộ Gel Rửa Mặt SVR Không Chứa Xà Phòng Cho Da ...
15053,3050,5,422205637,Mặt Nạ Dưỡng Da Some By Mi AHA-BHA-PHA Tẩy Tế ...


## 3. Đọc dữ liệu bằng reader

### đọc dữ liệu từ thư viện surprise

In [ ]:
reader = Reader(rating_scale=(1.0, 5.0))
data = Dataset.load_from_df(df_sub[['ma_khach_hang', 'ma_san_pham', 'so_sao']], reader)

## 4. Turning Model

### turning Model SVD

In [ ]:
from surprise.model_selection import RandomizedSearchCV
import random

In [ ]:
# Định nghĩa không gian siêu tham số
param_grid = {
    'n_factors': random.sample(range(50, 200), 5),  # Ngẫu nhiên 5 giá trị
    'n_epochs': [10, 20, 30],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.02, 0.05, 0.1]
}

# Tạo RandomizedSearchCV
random_search = RandomizedSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5, n_iter=10)

In [ ]:
# Huấn luyện
random_search.fit(data)

# Kết quả tốt nhất
print("Best RMSE:", random_search.best_score['rmse'])
print("Best Parameters:", random_search.best_params['rmse'])

Best RMSE: 0.6040363725064761
Best Parameters: {'n_factors': 136, 'n_epochs': 30, 'lr_all': 0.01, 'reg_all': 0.05}


### Turning Model KNN

In [ ]:
from surprise.model_selection import GridSearchCV

In [ ]:
param_grid = {
    'k': [10, 20, 30],             # Số lượng hàng xóm gần nhất
    'min_k': [1, 5, 10],           # Số lượng tối thiểu hàng xóm để dự đoán
    'sim_options': {
        'name': ['cosine', 'pearson'],
        'user_based': [True, False]  # Gợi ý dựa trên người dùng hoặc sản phẩm
    }
}

grid_search = GridSearchCV(KNNBasic, param_grid, measures=['rmse', 'mae'], cv=5)

In [ ]:
grid_search.fit(data)

print("Best RMSE:", grid_search.best_score['rmse'])
print("Best Parameters:", grid_search.best_params['rmse'])

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing

### Model SVD

In [ ]:
algorithm_SVD = SVD(n_factors = 136, n_epochs = 30,lr_all = 0.01, reg_all = 0.05)
# Run 5-fold cross-validation
result_SVD = cross_validate(algorithm_SVD, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.6051  0.5685  0.6491  0.5962  0.5867  0.6011  0.0269  
MAE (testset)     0.3013  0.2931  0.3234  0.2992  0.2906  0.3015  0.0116  
Fit time          0.55    0.55    0.55    0.53    0.55    0.55    0.01    
Test time         0.03    0.03    0.03    0.03    0.03    0.03    0.00    


### Model KNN

In [ ]:
algorithm_KNN = KNNBasic(k = 30, min_k=  1, sim_options = {'name': 'pearson', 'user_based': False})
# Run 5-fold cross-validation
result_KNN = cross_validate(algorithm_KNN, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.5447  0.5188  0.5470  0.5594  0.5509  0.5442  0.0136  
MAE (testset)     0.1593  0.1551  0.1705  0.1636  0.1694  0.1636  0.0059  
Fit time          0.02    0.02    0.02    0.02    0.02    0.02    0.00    
Test time         0.05    0.05    0.04    0.05    0.05    0.05    0.00    


### Nhận xét : ta thấy về RMSE , MAE , time Train, time test của mô hình KNN đều vượt trội hơn mô hình SVD , nên ta chọn model KNN

### Fit model

In [ ]:
# If the results are OK => getting full dataset => fit model
trainset = data.build_full_trainset()
algorithm_KNN.fit(trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


### Dự đoán

In [ ]:
pred = algorithm_KNN.predict(uid=443, iid=308500015,r_ui=5.0)
print(pred)

user: 443        item: 308500015  r_ui = 5.00   est = 5.00   {'actual_k': 11, 'was_impossible': False}


In [ ]:
# save model to disk
# import pickle
# with open("surprise_knn_model.pkl", "wb") as f:
#     pickle.dump(algorithm_KNN, f)

## 5 . Recommender for a specific user

### Các sản phẩm đã được mua từ khách hàng

In [ ]:
# userId = 443 has bought some products:
userId = 443
df_select = df_sub[(df_sub['ma_khach_hang'] == userId) & (df_sub['so_sao'] >=3)]
df_select = df_select.set_index('ma_san_pham')
df_select.head(df_select.shape[0])

,ma_khach_hang,so_sao,ten_san_pham
ma_san_pham,,,
308500015,443,5,"Mặt Nạ Naruko Bạch Ngọc Lan Dưỡng Sáng, Săn Ch..."
204100054,443,5,Gel Rửa Mặt Eucerin Cho Da Nhờn Mụn 400ml
422206537,443,5,Gel Rửa Mặt Eucerin Cho Da Nhờn Mụn 75ml
422202253,443,5,Mặt Nạ Naruko Ý Dĩ Nhân Đỏ Dưỡng Sáng Da 25ml ...
422205162,443,5,Hộp 10 Miếng Mặt Nạ Naruko Hoa Hồng Nhung Cấp ...
308500004,443,5,Mặt Nạ Naruko Tràm Trà Kiểm Soát Dầu Và Giảm M...
308500018,443,5,Mặt Nạ Naruko Hoa Thủy Tiên Hỗ Trợ Phục Hồi Da...
308500009,443,5,"Mặt Nạ Naruko Ý Dĩ Nhân Đỏ Se Lỗ Chân Lông, Sá..."
422205164,443,5,Hộp 10 Miếng Mặt Nạ Naruko Ý Dĩ Nhân Đỏ Sáng D...


In [ ]:
df_select.shape

(14, 3)

### hàm gợi ý 5 sản phẩm tốt nhất cho khách hàng

In [ ]:
import pandas as pd

In [ ]:
# The following are the top 5 product to be recommended to the user with userId
# To recommend products (i.e., product) to the given user,
# we can sort the list of movies in decreasing order of predicted ratings
# and take the top N product as recommendations:
def recommended_top5(userId,df_sub):
  df_score = df_sub[["ma_san_pham","ten_san_pham"]]

  #loại bỏ những sản phẩm đã được mua
  df_score = df_score[~df_score['ma_san_pham'].isin(df_select.index)]
  df_score.reset_index(drop=True,inplace=True)

  #tạo cột và dự đoán các giá trị
  df_score['EstimateScore'] = df_score['ma_san_pham'].apply(lambda x: algorithm_KNN.predict(userId, x).est) # est: get EstimateScore

  #sắp xếp lại các giá trị theo thứ tự giảm dần
  df_score = df_score.sort_values(by=['EstimateScore'], ascending=False)

  #xóa dữ liệu trùng
  df_score = df_score.drop_duplicates()

  #lọc những dòng có EstimateScore >= 4
  df_score = df_score[df_score['EstimateScore'] >= 4].head()
  return df_score

In [ ]:
recommended_top5(userId,df_sub) #userId = 443

,ma_san_pham,ten_san_pham,EstimateScore
0,422203809,Combo 3 Bông Tẩy Trang Ipek 150 Miếng,5.0
8882,200400003,Nước Tẩy Trang Bioderma Dành Cho Da Nhạy Cảm 5...,5.0
3438,422203146,Sữa Chống Nắng Sunplay Hiệu Chỉnh Sắc Da 50g (...,5.0
3439,205100146,Nước Tẩy Trang L'Oreal Làm Sạch Sâu Trang Điểm...,5.0
3444,249500013,Sữa Chống Nắng Sunplay Skin Aqua Dưỡng Da Sáng...,5.0
